<a href="https://colab.research.google.com/github/manabtikadar/my_project/blob/main/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import files
uploaded = files.upload()

Saving gender_submission.csv to gender_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [3]:
dataset = pd.read_csv('train.csv')
dataset.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
dataset.shape

(891, 12)

In [5]:
obj = (dataset.dtypes == 'object')
object_cols = list(obj[obj].index)
print(obj)
print("Categorical variables:",object_cols)
print('No. of. categorical features:',len(object_cols))

PassengerId    False
Survived       False
Pclass         False
Name            True
Sex             True
Age            False
SibSp          False
Parch          False
Ticket          True
Fare           False
Cabin           True
Embarked        True
dtype: bool
Categorical variables: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
No. of. categorical features: 5


In [6]:
numerical_dataset = dataset.select_dtypes(include=['int64', 'float64'])
print(numerical_dataset.head())

   PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare
0            1         0       3  22.0      1      0   7.2500
1            2         1       1  38.0      1      0  71.2833
2            3         1       3  26.0      0      0   7.9250
3            4         1       1  35.0      1      0  53.1000
4            5         0       3  35.0      0      0   8.0500


In [7]:
dataset.loc[dataset['Age'].isnull(),'Age']=np.round(dataset['Age'].mean())
dataset.loc[dataset['Embarked'].isnull(),'Embarked']=dataset['Embarked'].value_counts().index[0]

In [8]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [9]:
nb_train = int(np.floor(0.9 * len(dataset)))
dataset = dataset.sample(frac=1, random_state=217)
X_train = dataset[features][:nb_train]
y_train = dataset['Survived'][:nb_train].values
X_test = dataset[features][nb_train:]
y_test = dataset['Survived'][nb_train:].values

In [10]:
print(f"X_train:\n{X_train}\n")
print(f"y_train:\n{y_train}\n")
print(f"X_test:\n{X_test}\n")
print(f"y_test:\n{y_test}\n")

X_train:
     Pclass     Sex   Age  SibSp  Parch     Fare Embarked
593       3  female  30.0      0      2   7.7500        Q
56        2  female  21.0      0      0  10.5000        S
24        3  female   8.0      3      1  21.0750        S
222       3    male  51.0      0      0   8.0500        S
591       1  female  52.0      1      0  78.2667        C
..      ...     ...   ...    ...    ...      ...      ...
63        3    male   4.0      3      2  27.9000        S
855       3  female  18.0      0      1   9.3500        S
262       1    male  52.0      1      1  79.6500        S
8         3  female  27.0      0      2  11.1333        S
412       1  female  33.0      1      0  90.0000        Q

[801 rows x 7 columns]

y_train:
[0 1 0 0 1 0 0 1 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1
 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0
 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 0 0 0
 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0

# Bootstrapping
*One of the main reasons random forests are so powerul is due to the randomness injected into each tree. What do I mean by this? Each individual decision tree will be constructed on a bootstrapped subset of our data. If our dataset has n observations bootstrapping is the process of sampling n points with replacement. This means that some obsverations in our data set will be selected more than once and some won't be selected at all. We can actually calculate that the probability an observation is omitted from our bootstrapped dataset is (1−1/n)^n. By definition e^−1=limn→∞(1−1/n)^n and since e^−1 = 0.36787.. ≈1/3 ⇒ bootstrapping n samples with replacement will leave out approximately 1/3 of the observations in each distinct tree. Since each individual tree is built using only 2/3 of the data we'll find that most trees will differ significantly from one another. If interested.*

*The other great thing that comes with bootstrapping is that we get whats called an out-of-bag error estimate for free. The OOB (out-of-bag) samples are the ≈1/3 observations that were not selected to build a parcticular tree. Once we've built our tree with the n bootstrapped observations we can test each (xi->vector) that was left out and compute the mean prediction error from those samples. We can compute an OOB score for each tree and take the average of all those scores to get an estimate for how accurate our random forests performs, this is essentially leave-one-out cross validation. This will give us an estimate for how accurate our model is without having to formally test it on new data and we'll soon find that it is approximately the same error rate we'll get at test time.*



# Bagging
*The other great idea that random forests introduce is the concept of bagging. Bagging is the process of growing a tree where each node in the tree looks at every value in our bootstrapped sample in every feature to find the best split in the data at that particular node. This is repeated for all trees.*

*Random forests follow the same procedure as bagging, however, the key difference is that on a dataset with p features each tree will only look at a subset of m features where m=(√p). This is injecting even more randomness into the model due to the fact that if we sampled all p features in each tree we would likely be making splits at the same values from the same features in most trees. Given that we are only looking at (√p) features at one time many of the trees will look at a different groups of feature from one another. With this we'll be able to produce many uncorrelated trees which will help us capture a lot of the variability as well as interactions between multiple variables.*

# Random Forest Algorithm
*Suppose we have the following data data {(x⃗ 1,y1),(x⃗ 2,y2),..,(x⃗ n,yn)} where each x⃗ i represents a feature vector [xi1,xi2,...,xim] and let B be the number of trees we want to construct in our forest. We will do the following,*


1.   *for b=1 to B:*

*   *Draw a bootstrap sample of size n from the data*
*   *Grow a decision tree Tb from our bootstrapped sample, by repeating the following steps until the each node consists of 1 class only or until we've reached the minimum node size minsize specified beforehand*
        

    (i) sample m=p–√ features (where p is the number of features in our dataset)
    (ii) compute the information gain for each possible value among the bootstrapped data and m features
    (iii) split the node into 2 children nodes  
            





2.   *Output the ensemble of trees {T}B1*



In [11]:
def calculate_entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))

In [12]:
def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = calculate_entropy(p_parent)
    IG_l = calculate_entropy(p_left)
    IG_r = calculate_entropy(p_right)
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r

In [13]:
def draw_bootstrap(X_train,y_train):
  bootstrap_indices = list(np.random.choice(range(len(X_train)),len(X_train),replace=True))

  oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
  X_bootstrap = X_train.iloc[bootstrap_indices].values
  y_bootstrap = y_train[bootstrap_indices]
  X_oob = X_train.iloc[bootstrap_indices].values
  y_oob = y_train[bootstrap_indices]

  return X_bootstrap,y_bootstrap,X_oob,y_oob

*At each new iteration we'll use the OOB samples to evaluate the performance of the tree built with the bootstrapped data. So in other words, if we have 100 trees we'll have 100 OOB scores*

In [14]:
def oob_score(tree,X_test,y_test):
  mislabeled = 0
  for i in range(len(X_test)):
    pred = predict_tree(tree,X_test[i])
    if pred != y_test[i]:
      mislabeled += 1

  return mislabeled/len(X_test)

In [15]:
def find_split_point(X_bootstrap,y_bootstrap,max_features):
  features_ls = list()
  num_features = len(X_bootstrap[0])

  while len(features_ls) <= max_features:
    feature_idx = random.sample(range(num_features),1)
    if feature_idx not in features_ls:
      features_ls.extend(feature_idx)

  best_info_gain = -999
  node = None

  for feature_idx in features_ls:
   for split_point in X_bootstrap[:,feature_idx]:
      left_child = {'X_bootstrap': [],'y_bootstrap': []}
      right_child = {'X_bootstrap': [],'y_bootstrap': []}

      if type(split_point) in [int,float]:
        for i,value in enumerate(X_bootstrap[:,feature_idx]):
          if value <= split_point:
            left_child['X_bootstrap'].append(X_bootstrap[i])
            left_child['y_bootstrap'].append(y_bootstrap[i])
          else:
            right_child['X_bootstrap'].append(X_bootstrap[i])
            right_child['y_bootstrap'].append(y_bootstrap[i])
      else:
         for i,value in enumerate(X_bootstrap[:,feature_idx]):
           if value == split_point:
             left_child['X_bootstrap'].append(X_bootstrap[i])
             left_child['y_bootstrap'].append(y_bootstrap[i])
           else:
              right_child['X_bootstrap'].append(X_bootstrap[i])
              right_child['y_bootstrap'].append(y_bootstrap[i])

      split_info_gain = information_gain(left_child['y_bootstrap'],right_child['y_bootstrap'])

      if split_info_gain >= best_info_gain :
         best_info_gain = split_info_gain
         left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
         right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
         node = {'information_gain': split_info_gain,
                 'left_child': left_child,
                 'right_child': right_child,
                 'split_point': split_point,
                 'feature_idx': feature_idx}

  return node

# Terminal Node
We'll next need a function that decides when to stop splitting nodes in a tree and finally output a terminal node (classifies whether the passenger survives or perishes). On a single tree split_node works as follows:

1.   Given a node, store the left and right children as left_child & right_child and remove them from the original dictionary
2.   Check if either children has 0 observations in them. If one of the children is entirely empty this ultimately means that the best split in the data for that node was unable to differentiate the 2 classes and its best to call terminal_node and return the tree. terminal_node returns the class with the highest counts at the current node.
3.   Check if the current depth of the tree has reached the maximum depth. If so, create a terminal node and return the tree.
4.   Check if number of observations in the left child at the current node is less than the minimum samples needed to make a split, which will be stored as min_samples_split. If so create a terminal node and return the tree
5.   If the left child has more observations than min_samples_split we'll feed that left node into find_split_point again to find the best split point and repeat steps 1 - 6. This is ultimately going to be nesting dictionaries, which we are using to represent each node in our tree.
6.   Repeat steps 4 and 5 for the right child node
7.   Repeat steps 1 - 6 until each branch has a terminal node



In [16]:
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    if not y_bootstrap:  # If the node has no labels
        return None  # Or return a default class (e.g., 0 or -1)
    pred = max(y_bootstrap, key=y_bootstrap.count)
    return pred


In [17]:
def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    # Case 1: Handle empty child nodes (no data)
    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return node

    # Case 2: Max depth reached
    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    # Case 3: Min samples split reached
    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_features, min_samples_split, max_depth, depth + 1)

    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

# parameters:

1.   n_estimators: (int) The number of trees in the forest.
2.   max_features: (int) The number of features to consider when looking for the best split (typically √p)
3.   max_depth: (int) The maximum depth of the tree
4.   min_samples_split: (int) The minimum number of samples required to split an internal node

In [18]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node

In [19]:
def random_forest(X_train,y_train,max_depth,min_samples_split,n_estimators,max_features):
  tree_ls = list()
  oob_ls = list()

  for i in range(n_estimators):
    X_bootstrap,y_bootstrap,X_oob,y_oob = draw_bootstrap(X_train,y_train)
    tree = build_tree(X_bootstrap,y_bootstrap,max_depth,min_samples_split,max_features)
    tree_ls.append(tree)
    obb_error = oob_score(tree,X_oob,y_oob)
    oob_ls.append(obb_error)

  print(f"OOB estimate: {np.mean(oob_ls):.2f}")

  return tree_ls

In [20]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']

    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

In [21]:
def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)

In [22]:
n_estimators = 100
max_features = 3
max_depth = 10
min_samples_split = 2

model = random_forest(X_train, y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)

OOB estimate: 0.20


In [23]:
preds = predict_rf(model, X_test)
acc = sum(preds == y_test) / len(y_test)
print("Testing accuracy: {}".format(np.round(acc,3)))

Testing accuracy: 0.733


# Random Forest using library function

In [24]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , classification_report

In [43]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, max_features=3)

In [44]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Create a ColumnTransformer to one-hot encode categorical features
categorical_features = ['Sex', 'Embarked'] # List your categorical features
numerical_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] # List your numerical features

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),  # Passthrough numerical features
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features) # One-hot encode categorical features
    ])

# Fit and transform the training data
X_train_encoded = preprocessor.fit_transform(X_train)

# Transform the testing data (using the same encoding as training data)
X_test_encoded = preprocessor.transform(X_test)

# Now you can fit the model using the encoded data
rf.fit(X_train_encoded, y_train)

RandomForestClassifier(max_depth=10, max_features=3, min_samples_split=5)

In [45]:
y_pred = rf.predict(X_test_encoded)

In [46]:
print(y_pred)

[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0
 1 1 1 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1
 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0]


In [47]:
print(y_test)

[0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0
 0 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1
 0 1 1 0 0 1 1 0 0 0 0 1 0 1 1 0]


In [48]:
# Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Detailed classification metrics
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8555555555555555
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.94      0.88        53
           1       0.90      0.73      0.81        37

    accuracy                           0.86        90
   macro avg       0.87      0.84      0.85        90
weighted avg       0.86      0.86      0.85        90



In [49]:
importances = rf.feature_importances_
for i, feature in enumerate(features): # Change dataset.feature_names to dataset.columns
    print(f"{feature}: {importances[i]:.4f}")

Pclass: 0.0984
Sex: 0.1793
Age: 0.0512
SibSp: 0.0378
Parch: 0.2206
Fare: 0.2232
Embarked: 0.1465


In [40]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,  # 5-fold cross-validation
    verbose=2
)

# Fit the model
grid_search.fit(X_train_encoded, y_train)

# Best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate on test data
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_encoded)
print("Accuracy with Best Hyperparameters:", accuracy_score(y_test, y_pred_best))


Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=50; total time=   0.2s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   1.1s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END max_depth=None, min_s

In [50]:
preds = rf.predict(X_test_encoded)
acc = sum(preds == y_test) / len(y_test)
print("Testing accuracy: {}".format(np.round(acc,3)))

Testing accuracy: 0.856
